In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import time

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.path import Path
from multiprocess import Pool
import pandas
import cv2

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from data_manager import *
from metadata import *
from cell_utilities import *
from learning_utilities import *

Setting environment for Gordon


No vtk
Cannot import mxnet.


# One stack - all subregions

In [2]:
patch_size = 448
stride = 224
half_size = patch_size/2

In [ ]:
for stack in all_annotated_stacks:
# for stack in ['MD589']:

    first_bs_section, last_bs_section = metadata_cache['section_limits'][stack]
    grid_spec=(patch_size, stride, ) + metadata_cache['image_shape'][stack]
    grid_locations = grid_parameters_to_sample_locations(grid_spec)

    for sec in range(first_bs_section, last_bs_section+1):
#     for sec in range(204, 205):

        if is_invalid(sec=sec, stack=stack):
            continue
            
        mask_tb = DataManager.load_thumbnail_mask_v2(stack, sec)
        region_patch_indices = locate_patches_v2(stack=stack, grid_spec=grid_spec, mask_tb=mask_tb)
        
        region_patch_locations = grid_locations[region_patch_indices]

        ul = region_patch_locations + (-half_size, -half_size)
        ur = region_patch_locations + (-half_size, half_size)
        lr = region_patch_locations + (half_size, half_size)
        ll = region_patch_locations + (half_size, -half_size)
        
        region_contours = np.array(zip(ul, ur, lr, ll))
        
        t = time.time()
        compute_features_regions(stack, sec, region_contours)
        sys.stderr.write('Time: %d seconds\n' % (time.time()-t)) # 257 seconds
        
        locations_fp = get_cell_classifier_data_filepath('region_locations', stack=stack, sec=sec, ext='txt')
        np.savetxt(locations_fp, np.c_[region_patch_indices, region_patch_locations], fmt='%d %d %d')

1875 patches in fg
Processing stack MD585, section 83...
Neighbor search: 5.16 seconds


# One stack - sampled subregions

In [ ]:
patch_size = 448
stride = 56
half_size = patch_size/2

In [ ]:
training_contours_all_labels = defaultdict(list)
training_addresses_all_labels = defaultdict(list)

training_contours_all_labels_per_section = defaultdict(lambda: defaultdict(list))
training_addresses_all_labels_per_section = defaultdict(lambda: defaultdict(list))

for stack in all_annotated_stacks:

    first_bs_section, last_bs_section = metadata_cache['section_limits'][stack]
    grid_spec=(patch_size, stride, ) + metadata_cache['image_shape'][stack]
    grid_locations = grid_parameters_to_sample_locations(grid_spec)

    for sec in range(first_bs_section, last_bs_section+1):
#     for sec in range(200, 205):
        print stack, sec

        if is_invalid(sec=sec, stack=stack):
            continue

        patch_indices_df = locate_annotated_patches_v2(stack=stack, grid_spec=grid_spec, sections=[sec],
                                                      surround_margins=[200, 500])
        if len(patch_indices_df) == 0:
            continue

        for label, indices in patch_indices_df[sec].iteritems():
            if len(indices) == 0:
                continue
            random_indices = np.random.choice(indices, min(10, len(indices)), replace=False)
            random_locations = grid_locations[random_indices]
            for l, i in zip(random_locations, random_indices):
                cnt = np.array([l + (-half_size, -half_size), l + (-half_size, half_size), \
                                l + (half_size, half_size), l + (half_size, -half_size)])

                training_contours_all_labels[label].append(cnt)
                training_addresses_all_labels[label].append((stack, sec, i))

                training_contours_all_labels_per_section[(stack, sec)][label].append(cnt)
                training_addresses_all_labels_per_section[(stack, sec)][label].append(i)
                
        compute_features_per_section(stack, sec, training_contours_all_labels_per_section[(stack, sec)], 
                                     center_structures=None)

    #         patches = extract_patches_given_locations(stack, sec, grid_spec=grid_spec, indices=random_indices)

training_contours_all_labels.default_factory = None
training_addresses_all_labels.default_factory = None

training_contours_all_labels_per_section.default_factory = None
training_addresses_all_labels_per_section.default_factory = None

In [ ]:
# center_structures = ['RtTg']
center_structures = None

In [ ]:
for stack, sec in sorted(training_contours_all_labels_per_section.keys()):
# for stack, sec in [('MD589', 203)]:
    print stack, sec
    if stack != 'MD594':
        continue
    
    training_contours_all_labels = training_contours_all_labels_per_section[(stack, sec)]

    sys.stderr.write('Processing section %d...\n' % sec)

    cell_orientations = load_cell_data('orientation', stack, sec)
    cell_orientations = np.array(map(normalize_angle, cell_orientations))

    cell_centroids = load_cell_data('centroid', stack, sec)
    cell_numbers = cell_centroids.shape[0]

    def flatten_cells(what, sec):
        cells = load_cell_data(what, stack, sec)
        cells = cells.reshape((cells.shape[0], -1))
        return cells

    cells = flatten_cells('cells_aligned_padded', sec)
    cells_h = flatten_cells('cells_aligned_padded_h', sec)
    cells_v = flatten_cells('cells_aligned_padded_v', sec)
    cells_d = flatten_cells('cells_aligned_padded_d', sec)

    cell_sizes = np.sum(cells, axis=1)

    large_cell_threshold = 163
    large_cell_indices = np.where(cell_sizes > large_cell_threshold)[0]
    large_cell_centroids = cell_centroids[large_cell_indices]

    #####################################
    # Find each LARGE cell's neighbors
    #####################################

    from scipy.spatial.kdtree import KDTree
    from scipy.spatial.distance import cdist, pdist

    t = time.time()

    tree = KDTree(cell_centroids)

    pool = Pool(12)
    neighbors = pool.map(lambda i: list(set(tree.query_ball_point(cell_centroids[i], r=100)) - {i}),
                         large_cell_indices)
    pool.terminate()
    pool.join()

    sys.stderr.write('Neighbor search: %.2f seconds\n' % (time.time()-t)) # 10 seconds

    neighbors = dict(zip(large_cell_indices, neighbors))

    # Compute neighbot vectors

    neighbor_vectors = {i: cell_centroids[i] - cell_centroids[nns] 
                        for i, nns in neighbors.iteritems()}

    # Binning each cell's neighbors

    radial_indices_all = {}
    angular_indices_all = {}
    for i in neighbor_vectors.iterkeys():
        radial_indices, angular_indices = allocate_radial_angular_bins(neighbor_vectors[i], 
                                                                       cell_orientations[i],
                                                            angular_bins=angular_bins, radial_bins=radial_bins)
        radial_indices_all[i] = radial_indices
        angular_indices_all[i] = angular_indices

    ##############################

    neighbor_info = {'neighbors': neighbors,
     'neighbor_vectors': neighbor_vectors,
     'radial_indices': radial_indices_all,
     'angular_indices': angular_indices_all
    }

    fp = get_cell_classifier_data_filepath('training_regions_neighbor_info', stack, sec, ext='pkl')
    create_if_not_exists(os.path.dirname(fp))
    save_pickle(neighbor_info, fp)

    #     contours = {cnt['name']: cnt['vertices'] for cnt_id, cnt in contour_df[contour_df['section'] == sec].iterrows()}

    #############################
    # Compute histograms
    #############################

    all_cell_size_histogram_all_labels_all_regions = defaultdict(list)
    all_cell_size_weighted_histogram_all_labels_all_regions = defaultdict(list)
    large_cell_size_histogram_all_labels_all_regions = defaultdict(list)

    large_cell_orientation_histogram_all_labels_all_regions = defaultdict(list)

    neighbor_distance_histogram_all_labels_all_regions = defaultdict(list)
    neighbor_direction_histogram_all_labels_all_regions = defaultdict(list)

    large_cell_graph_neighbor_distance_histogram_all_labels_all_regions = defaultdict(list)
    large_cell_graph_neighbor_direction_histogram_all_labels_all_regions = defaultdict(list)

    large_small_cell_graph_neighbor_distance_histogram_all_labels_all_regions = defaultdict(list)
    large_small_cell_graph_neighbor_direction_histogram_all_labels_all_regions = defaultdict(list)

    #######################################################
    # Identify large and small cells
    ######################################################
    
    t = time.time()

    for name_u, cnts in training_contours_all_labels.iteritems():

        if center_structures is not None:
            if convert_to_original_name(name_u) not in center_structures:
                continue

        for cnt in cnts:

            # Get large cells

            large_cell_is_inside = Path(cnt.astype(np.int)).contains_points(large_cell_centroids)
            large_cell_indices_inside = large_cell_indices[large_cell_is_inside]

    #         print '%d large cells are identified in %s.' % (len(large_cell_indices_inside), name_u)

            # Small cells

            small_cell_indices = np.array(list(set(range(cell_numbers)) - set(large_cell_indices.tolist())))
            small_cell_centroids = cell_centroids[small_cell_indices]
            small_cell_is_inside = Path(cnt.astype(np.int)).contains_points(small_cell_centroids)
            small_cell_indices_inside = small_cell_indices[small_cell_is_inside]

    #         print '%d small cells are identified in %s.' % (len(small_cell_indices_inside), name_u)

            #################################
            # Compute cell size distribution
            #################################

            large_cell_size_inside_histogram, _ = np.histogram([cell_sizes[i] for i in large_cell_indices_inside], bins=size_bins)
            large_cell_size_histogram_all_labels_all_regions[name_u].append(large_cell_size_inside_histogram)

            all_cell_size_inside_histogram, _ = np.histogram([cell_sizes[i] for i in np.r_[large_cell_indices_inside, small_cell_indices_inside]], 
                                                   bins=size_bins)
            all_cell_size_histogram_all_labels_all_regions[name_u].append(all_cell_size_inside_histogram)


            all_cell_size_weighted_inside_histogram, _ = np.histogram([cell_sizes[i] for i in np.r_[large_cell_indices_inside, small_cell_indices_inside]], 
                                               bins=size_bins, 
                                                         weights=[cell_sizes[i] for i in np.r_[large_cell_indices_inside, small_cell_indices_inside]])
            all_cell_size_weighted_histogram_all_labels_all_regions[name_u].append(all_cell_size_weighted_inside_histogram)


            #########################################
            # Compute cell orientation distribution
            #########################################

            large_cell_orientation_inside_histogram, _ = np.histogram([cell_orientations[i] for i in large_cell_indices_inside], bins=orientation_bins)
        #     print 'large_cell_orientation_inside_histogram:', large_cell_orientation_inside_histogram

            large_cell_orientation_histogram_all_labels_all_regions[name_u].append(large_cell_orientation_inside_histogram)

            ##########################################
            # Construct graph
            # Start from large cells, link to all cells
            ##########################################

            import networkx as nx
            g = nx.Graph()

            for source_sectionwise_idx in large_cell_indices_inside:

                if len(neighbors[source_sectionwise_idx]) == 0:
                    continue

                neighbor_masks = np.array([cells[i_sectionwise_idx]
                                  for i_sectionwise_idx in neighbors[source_sectionwise_idx]])

                jacs, _ = compute_jaccard_x_vs_list(cells[source_sectionwise_idx], 
                                                neighbor_masks,
                                                x_h=cells_h[source_sectionwise_idx],
                                                 x_v=cells_v[source_sectionwise_idx],
                                                 x_d=cells_d[source_sectionwise_idx])

                for i_sectionwise_idx, vec, jac in zip(neighbors[source_sectionwise_idx], neighbor_vectors[source_sectionwise_idx], jacs):
                    length = np.sqrt(np.sum(vec**2))
                    direction = np.arctan2(vec[1], vec[0])

                    # Force into between -np.pi/2 and np.pi/2
                    if direction > np.pi/2:
                        direction = direction - np.pi
                    elif direction < -np.pi/2:
                        direction = direction + np.pi

                    size_diff = np.abs(cell_sizes[source_sectionwise_idx] - cell_sizes[i_sectionwise_idx])

                    orientation_diff = cell_orientations[source_sectionwise_idx] - cell_orientations[i_sectionwise_idx]
                    if orientation_diff > np.pi/2:
                        orientation_diff -= np.pi
                    elif orientation_diff < -np.pi/2:
                        orientation_diff += np.pi

                    g.add_edge(source_sectionwise_idx, i_sectionwise_idx, weight=1, length=length, direction=direction, 
                               orientation_diff=orientation_diff,
                              size_diff=size_diff,
                              jaccard=jac)

            ##########################################
            # Collect edge attributes
            ##########################################

            all_edge_length = []
            all_edge_direction = []
            for u, v in g.edges_iter():
            #     print u, v
                ed = g.get_edge_data(u, v)
                all_edge_length.append(ed['length'])
                all_edge_direction.append(ed['direction'])

            ##########################################
            # Compute neighbor distance histogram
            ##########################################

        #     print 'min', np.min(all_edge_length), 'max', np.max(all_edge_length)

            edge_length_histogram, _ = np.histogram(all_edge_length, bins=edge_length_bins)
            neighbor_distance_histogram_all_labels_all_regions[name_u].append(edge_length_histogram)

            ##########################################
            # Compute neighbor direction histogram
            ##########################################

            edge_direction_histogram, _ = np.histogram(all_edge_direction, bins=edge_direction_bins)
            neighbor_direction_histogram_all_labels_all_regions[name_u].append(edge_direction_histogram)

            ################################
            # Subgraph between large cells.
            ################################

            large_cell_subgraph = g.subgraph(large_cell_indices_inside)

            all_large_cell_graph_edge_length = []
            all_large_cell_graph_edge_direction = []
            for u, v in large_cell_subgraph.edges_iter():
                ed = large_cell_subgraph.get_edge_data(u, v)
                all_large_cell_graph_edge_length.append(ed['length'])
                all_large_cell_graph_edge_direction.append(ed['direction'])

            ##########################################
            # Compute neighbor distance histogram
            ##########################################

        #     print 'min', np.min(all_edge_length), 'max', np.max(all_edge_length)

            edge_length_histogram, _ = np.histogram(all_large_cell_graph_edge_length, bins=edge_length_bins)
            large_cell_graph_neighbor_distance_histogram_all_labels_all_regions[name_u].append(edge_length_histogram)

            ##########################################
            # Compute neighbor direction histogram
            ##########################################

            edge_direction_histogram, _ = np.histogram(all_large_cell_graph_edge_direction, bins=edge_direction_bins)
            large_cell_graph_neighbor_direction_histogram_all_labels_all_regions[name_u].append(edge_direction_histogram)

            ##########################################
            # Large/small cells graph
            ##########################################

            all_large_small_cell_graph_edge_length = []
            all_large_small_cell_graph_edge_direction = []

            for u, v in g.edges_iter():
                if (u in large_cell_indices_inside and v in small_cell_indices_inside) or \
                (v in large_cell_indices_inside and u in small_cell_indices_inside):
                    ed = g.get_edge_data(u,v)
                    all_large_small_cell_graph_edge_length.append(ed['length'])
                    all_large_small_cell_graph_edge_direction.append(ed['direction'])

            edge_length_histogram, _ = np.histogram(all_large_small_cell_graph_edge_length, bins=edge_length_bins)
            large_small_cell_graph_neighbor_distance_histogram_all_labels_all_regions[name_u].append(edge_length_histogram)

            edge_direction_histogram, _ = np.histogram(all_large_small_cell_graph_edge_direction, bins=edge_direction_bins)
            large_small_cell_graph_neighbor_direction_histogram_all_labels_all_regions[name_u].append(edge_direction_histogram)

    sys.stderr.write('Graph features: %.2f seconds\n' % (time.time()-t)) # 10 seconds
            
    region_features = \
    {'all_cell_size_hist': all_cell_size_histogram_all_labels_all_regions,
    'all_cell_size_weighted_hist': all_cell_size_weighted_histogram_all_labels_all_regions,
    'large_cell_size_hist': large_cell_size_histogram_all_labels_all_regions,
    'large_cell_orientation_hist': large_cell_orientation_histogram_all_labels_all_regions,
    'neighbor_distance_hist': neighbor_distance_histogram_all_labels_all_regions,
    'neighbor_direction_hist': neighbor_direction_histogram_all_labels_all_regions,
    'large_cell_graph_neighbor_distance_hist': large_cell_graph_neighbor_distance_histogram_all_labels_all_regions,
    'large_cell_graph_neighbor_direction_hist': large_cell_graph_neighbor_direction_histogram_all_labels_all_regions,
    'large_small_cell_graph_neighbor_distance_hist': large_small_cell_graph_neighbor_distance_histogram_all_labels_all_regions,
    'large_small_cell_graph_neighbor_direction_hist': large_small_cell_graph_neighbor_direction_histogram_all_labels_all_regions}

    fp = get_cell_classifier_data_filepath('training_regions_region_features', stack, sec, ext='pkl')
    create_if_not_exists(os.path.dirname(fp))
    save_pickle(region_features, fp)

In [ ]:
compute_features_per_section('MD589', 203, 
                             training_contours_all_labels_per_section[('MD589', 203)], 
                             center_structures=['RtTg'])

In [ ]:
tree = None # important, otherwise multiprocess.Pool does not work

In [ ]:
def compute_features_per_section(stack, sec, training_contours_all_labels, center_structures=None):
    
    print stack, sec

    sys.stderr.write('Processing section %d...\n' % sec)

    cell_orientations = load_cell_data('orientation', stack, sec)
    cell_orientations = np.array(map(normalize_angle, cell_orientations))

    cell_centroids = load_cell_data('centroid', stack, sec)
    cell_numbers = cell_centroids.shape[0]

    def flatten_cells(what, sec):
        cells = load_cell_data(what, stack, sec)
        cells = cells.reshape((cells.shape[0], -1))
        return cells

    cells = flatten_cells('cells_aligned_padded', sec)
    cells_h = flatten_cells('cells_aligned_padded_h', sec)
    cells_v = flatten_cells('cells_aligned_padded_v', sec)
    cells_d = flatten_cells('cells_aligned_padded_d', sec)

    cell_sizes = np.sum(cells, axis=1)

    large_cell_threshold = 163
    large_cell_indices = np.where(cell_sizes > large_cell_threshold)[0]
    large_cell_centroids = cell_centroids[large_cell_indices]

    #####################################
    # Find each LARGE cell's neighbors
    #####################################

    from scipy.spatial.kdtree import KDTree
    from scipy.spatial.distance import cdist, pdist

    t = time.time()

    global tree
    tree = KDTree(cell_centroids)

    pool = Pool(12)
    neighbors = pool.map(lambda i: list(set(tree.query_ball_point(cell_centroids[i], r=100)) - {i}),
                         large_cell_indices)
    pool.terminate()
    pool.join()

    sys.stderr.write('Neighbor search: %.2f seconds\n' % (time.time()-t)) # 10 seconds

    neighbors = dict(zip(large_cell_indices, neighbors))

    # Compute neighbot vectors

    neighbor_vectors = {i: cell_centroids[i] - cell_centroids[nns] 
                        for i, nns in neighbors.iteritems()}

    # Binning each cell's neighbors

    radial_indices_all = {}
    angular_indices_all = {}
    for i in neighbor_vectors.iterkeys():
        radial_indices, angular_indices = allocate_radial_angular_bins(neighbor_vectors[i], 
                                                                       cell_orientations[i],
                                                            angular_bins=angular_bins, radial_bins=radial_bins)
        radial_indices_all[i] = radial_indices
        angular_indices_all[i] = angular_indices

    ##############################

    neighbor_info = {'neighbors': neighbors,
     'neighbor_vectors': neighbor_vectors,
     'radial_indices': radial_indices_all,
     'angular_indices': angular_indices_all
    }
    
    fp = get_cell_classifier_data_filepath('neighbor_info', stack, sec, ext='pkl')
    create_if_not_exists(os.path.dirname(fp))
    save_pickle(neighbor_info, fp)

#     contours = {cnt['name']: cnt['vertices'] for cnt_id, cnt in contour_df[contour_df['section'] == sec].iterrows()}

    #############################
    # Compute histograms
    #############################

    all_cell_size_histogram_all_labels_all_regions = defaultdict(list)
    all_cell_size_weighted_histogram_all_labels_all_regions = defaultdict(list)
    large_cell_size_histogram_all_labels_all_regions = defaultdict(list)

    large_cell_orientation_histogram_all_labels_all_regions = defaultdict(list)

    neighbor_distance_histogram_all_labels_all_regions = defaultdict(list)
    neighbor_direction_histogram_all_labels_all_regions = defaultdict(list)

    large_cell_graph_neighbor_distance_histogram_all_labels_all_regions = defaultdict(list)
    large_cell_graph_neighbor_direction_histogram_all_labels_all_regions = defaultdict(list)

    large_small_cell_graph_neighbor_distance_histogram_all_labels_all_regions = defaultdict(list)
    large_small_cell_graph_neighbor_direction_histogram_all_labels_all_regions = defaultdict(list)

    #######################################################
    # Identify large and small cells
    ######################################################

    for name_u, cnts in training_contours_all_labels.iteritems():
        
        if center_structures is not None:
            if convert_to_original_name(name_u) not in center_structures:
                continue

        for cnt in cnts:

            # Get large cells

            large_cell_is_inside = Path(cnt.astype(np.int)).contains_points(large_cell_centroids)
            large_cell_indices_inside = large_cell_indices[large_cell_is_inside]

    #         print '%d large cells are identified in %s.' % (len(large_cell_indices_inside), name_u)

            # Small cells

            small_cell_indices = np.array(list(set(range(cell_numbers)) - set(large_cell_indices.tolist())))
            small_cell_centroids = cell_centroids[small_cell_indices]
            small_cell_is_inside = Path(cnt.astype(np.int)).contains_points(small_cell_centroids)
            small_cell_indices_inside = small_cell_indices[small_cell_is_inside]

    #         print '%d small cells are identified in %s.' % (len(small_cell_indices_inside), name_u)

            #################################
            # Compute cell size distribution
            #################################

            large_cell_size_inside_histogram, _ = np.histogram([cell_sizes[i] for i in large_cell_indices_inside], bins=size_bins)
            large_cell_size_histogram_all_labels_all_regions[name_u].append(large_cell_size_inside_histogram)

            all_cell_size_inside_histogram, _ = np.histogram([cell_sizes[i] for i in np.r_[large_cell_indices_inside, small_cell_indices_inside]], 
                                                   bins=size_bins)
            all_cell_size_histogram_all_labels_all_regions[name_u].append(all_cell_size_inside_histogram)


            all_cell_size_weighted_inside_histogram, _ = np.histogram([cell_sizes[i] for i in np.r_[large_cell_indices_inside, small_cell_indices_inside]], 
                                               bins=size_bins, 
                                                         weights=[cell_sizes[i] for i in np.r_[large_cell_indices_inside, small_cell_indices_inside]])
            all_cell_size_weighted_histogram_all_labels_all_regions[name_u].append(all_cell_size_weighted_inside_histogram)


            #########################################
            # Compute cell orientation distribution
            #########################################

            large_cell_orientation_inside_histogram, _ = np.histogram([cell_orientations[i] for i in large_cell_indices_inside], bins=orientation_bins)
        #     print 'large_cell_orientation_inside_histogram:', large_cell_orientation_inside_histogram

            large_cell_orientation_histogram_all_labels_all_regions[name_u].append(large_cell_orientation_inside_histogram)

            ##########################################
            # Construct graph
            # Start from large cells, link to all cells
            ##########################################

            import networkx as nx
            g = nx.Graph()

            for source_sectionwise_idx in large_cell_indices_inside:

                if len(neighbors[source_sectionwise_idx]) == 0:
                    continue

                neighbor_masks = np.array([cells[i_sectionwise_idx]
                                  for i_sectionwise_idx in neighbors[source_sectionwise_idx]])

                jacs, _ = compute_jaccard_x_vs_list(cells[source_sectionwise_idx], 
                                                neighbor_masks,
                                                x_h=cells_h[source_sectionwise_idx],
                                                 x_v=cells_v[source_sectionwise_idx],
                                                 x_d=cells_d[source_sectionwise_idx])

                for i_sectionwise_idx, vec, jac in zip(neighbors[source_sectionwise_idx], neighbor_vectors[source_sectionwise_idx], jacs):
                    length = np.sqrt(np.sum(vec**2))
                    direction = np.arctan2(vec[1], vec[0])

                    # Force into between -np.pi/2 and np.pi/2
                    if direction > np.pi/2:
                        direction = direction - np.pi
                    elif direction < -np.pi/2:
                        direction = direction + np.pi

                    size_diff = np.abs(cell_sizes[source_sectionwise_idx] - cell_sizes[i_sectionwise_idx])

                    orientation_diff = cell_orientations[source_sectionwise_idx] - cell_orientations[i_sectionwise_idx]
                    if orientation_diff > np.pi/2:
                        orientation_diff -= np.pi
                    elif orientation_diff < -np.pi/2:
                        orientation_diff += np.pi

                    g.add_edge(source_sectionwise_idx, i_sectionwise_idx, weight=1, length=length, direction=direction, 
                               orientation_diff=orientation_diff,
                              size_diff=size_diff,
                              jaccard=jac)

            ##########################################
            # Collect edge attributes
            ##########################################

            all_edge_length = []
            all_edge_direction = []
            for u, v in g.edges_iter():
            #     print u, v
                ed = g.get_edge_data(u, v)
                all_edge_length.append(ed['length'])
                all_edge_direction.append(ed['direction'])

            ##########################################
            # Compute neighbor distance histogram
            ##########################################

        #     print 'min', np.min(all_edge_length), 'max', np.max(all_edge_length)

            edge_length_histogram, _ = np.histogram(all_edge_length, bins=edge_length_bins)
            neighbor_distance_histogram_all_labels_all_regions[name_u].append(edge_length_histogram)

            ##########################################
            # Compute neighbor direction histogram
            ##########################################

            edge_direction_histogram, _ = np.histogram(all_edge_direction, bins=edge_direction_bins)
            neighbor_direction_histogram_all_labels_all_regions[name_u].append(edge_direction_histogram)

            ################################
            # Subgraph between large cells.
            ################################

            large_cell_subgraph = g.subgraph(large_cell_indices_inside)

            all_large_cell_graph_edge_length = []
            all_large_cell_graph_edge_direction = []
            for u, v in large_cell_subgraph.edges_iter():
                ed = large_cell_subgraph.get_edge_data(u, v)
                all_large_cell_graph_edge_length.append(ed['length'])
                all_large_cell_graph_edge_direction.append(ed['direction'])

            ##########################################
            # Compute neighbor distance histogram
            ##########################################

        #     print 'min', np.min(all_edge_length), 'max', np.max(all_edge_length)

            edge_length_histogram, _ = np.histogram(all_large_cell_graph_edge_length, bins=edge_length_bins)
            large_cell_graph_neighbor_distance_histogram_all_labels_all_regions[name_u].append(edge_length_histogram)

            ##########################################
            # Compute neighbor direction histogram
            ##########################################

            edge_direction_histogram, _ = np.histogram(all_large_cell_graph_edge_direction, bins=edge_direction_bins)
            large_cell_graph_neighbor_direction_histogram_all_labels_all_regions[name_u].append(edge_direction_histogram)

            ##########################################
            # Large/small cells graph
            ##########################################

            all_large_small_cell_graph_edge_length = []
            all_large_small_cell_graph_edge_direction = []

            for u, v in g.edges_iter():
                if (u in large_cell_indices_inside and v in small_cell_indices_inside) or \
                (v in large_cell_indices_inside and u in small_cell_indices_inside):
                    ed = g.get_edge_data(u,v)
                    all_large_small_cell_graph_edge_length.append(ed['length'])
                    all_large_small_cell_graph_edge_direction.append(ed['direction'])

            edge_length_histogram, _ = np.histogram(all_large_small_cell_graph_edge_length, bins=edge_length_bins)
            large_small_cell_graph_neighbor_distance_histogram_all_labels_all_regions[name_u].append(edge_length_histogram)

            edge_direction_histogram, _ = np.histogram(all_large_small_cell_graph_edge_direction, bins=edge_direction_bins)
            large_small_cell_graph_neighbor_direction_histogram_all_labels_all_regions[name_u].append(edge_direction_histogram)
    
    region_features = \
    {'all_cell_size_hist': all_cell_size_histogram_all_labels_all_regions,
    'all_cell_size_weighted_hist': all_cell_size_weighted_histogram_all_labels_all_regions,
    'large_cell_size_hist': large_cell_size_histogram_all_labels_all_regions,
    'large_cell_orientation_hist': large_cell_orientation_histogram_all_labels_all_regions,
    'neighbor_distance_hist': neighbor_distance_histogram_all_labels_all_regions,
    'neighbor_direction_hist': neighbor_direction_histogram_all_labels_all_regions,
    'large_cell_graph_neighbor_distance_hist': large_cell_graph_neighbor_distance_histogram_all_labels_all_regions,
    'large_cell_graph_neighbor_direction_hist': large_cell_graph_neighbor_direction_histogram_all_labels_all_regions,
    'large_small_cell_graph_neighbor_distance_hist': large_small_cell_graph_neighbor_distance_histogram_all_labels_all_regions,
    'large_small_cell_graph_neighbor_direction_hist': large_small_cell_graph_neighbor_direction_histogram_all_labels_all_regions}
    
    fp = get_cell_classifier_data_filepath('region_features', stack, sec, ext='pkl')
    create_if_not_exists(os.path.dirname(fp))
    save_pickle(region_features, fp)
            
#     all_cell_size_histogram_all_labels_all_regions_per_section[(stack, sec)] = all_cell_size_histogram_all_labels_all_regions
#     all_cell_size_weighted_histogram_all_labels_all_regions_per_section[(stack, sec)] = all_cell_size_weighted_histogram_all_labels_all_regions
#     large_cell_size_histogram_all_labels_all_regions_per_section[(stack, sec)] = large_cell_size_histogram_all_labels_all_regions

#     large_cell_orientation_histogram_all_labels_all_regions_per_section[(stack, sec)] = large_cell_orientation_histogram_all_labels_all_regions
#     neighbor_distance_histogram_all_labels_all_regions_per_section[(stack, sec)] = neighbor_distance_histogram_all_labels_all_regions
#     neighbor_direction_histogram_all_labels_all_regions_per_section[(stack, sec)] = neighbor_direction_histogram_all_labels_all_regions

#     large_cell_graph_neighbor_distance_histogram_all_labels_all_regions_per_section[(stack, sec)] = large_cell_graph_neighbor_distance_histogram_all_labels_all_regions
#     large_cell_graph_neighbor_direction_histogram_all_labels_all_regions_per_section[(stack, sec)] = large_cell_graph_neighbor_direction_histogram_all_labels_all_regions

#     large_small_cell_graph_neighbor_distance_histogram_all_labels_all_regions_per_section[(stack, sec)] = large_small_cell_graph_neighbor_distance_histogram_all_labels_all_regions
#     large_small_cell_graph_neighbor_direction_histogram_all_labels_all_regions_per_section[(stack, sec)] = large_small_cell_graph_neighbor_direction_histogram_all_labels_all_regions


In [ ]:
all_cell_size_histogram_all_labels_all_regions_per_section = {}
all_cell_size_weighted_histogram_all_labels_all_regions_per_section = {}
large_cell_size_histogram_all_labels_all_regions_per_section = {}

large_cell_orientation_histogram_all_labels_all_regions_per_section = {}
neighbor_distance_histogram_all_labels_all_regions_per_section = {}
neighbor_direction_histogram_all_labels_all_regions_per_section = {}

large_cell_graph_neighbor_distance_histogram_all_labels_all_regions_per_section = {}
large_cell_graph_neighbor_direction_histogram_all_labels_all_regions_per_section = {}

large_small_cell_graph_neighbor_distance_histogram_all_labels_all_regions_per_section = {}
large_small_cell_graph_neighbor_direction_histogram_all_labels_all_regions_per_section = {}

In [ ]:
for stack, sec in sorted(training_contours_all_labels_per_section.keys()):
    
    print stack, sec

    sys.stderr.write('Processing section %d...\n' % sec)

    cell_orientations = load_cell_data('orientation', stack, sec)
    cell_orientations = np.array(map(normalize_angle, cell_orientations))

    cell_centroids = load_cell_data('centroid', stack, sec)
    cell_numbers = cell_centroids.shape[0]

    def flatten_cells(what, sec):
        cells = load_cell_data(what, stack, sec)
        cells = cells.reshape((cells.shape[0], -1))
        return cells

    cells = flatten_cells('cells_aligned_padded', sec)
    cells_h = flatten_cells('cells_aligned_padded_h', sec)
    cells_v = flatten_cells('cells_aligned_padded_v', sec)
    cells_d = flatten_cells('cells_aligned_padded_d', sec)

    cell_sizes = np.sum(cells, axis=1)

    large_cell_threshold = 163
    large_cell_indices = np.where(cell_sizes > large_cell_threshold)[0]
    large_cell_centroids = cell_centroids[large_cell_indices]

    #####################################
    # Find each LARGE cell's neighbors
    #####################################

    from scipy.spatial.kdtree import KDTree
    from scipy.spatial.distance import cdist, pdist

    t = time.time()

    tree = KDTree(cell_centroids)

    pool = Pool(12)
    neighbors = pool.map(lambda i: list(set(tree.query_ball_point(cell_centroids[i], r=100)) - {i}),
                         large_cell_indices)
    pool.terminate()
    pool.join()

    sys.stderr.write('Neighbor search: %.2f seconds\n' % (time.time()-t)) # 10 seconds

    neighbors = dict(zip(large_cell_indices, neighbors))

    # Compute neighbot vectors

    neighbor_vectors = {i: cell_centroids[i] - cell_centroids[nns] 
                        for i, nns in neighbors.iteritems()}

    # Binning each cell's neighbors

    radial_indices_all = {}
    angular_indices_all = {}
    for i in neighbor_vectors.iterkeys():
        radial_indices, angular_indices = allocate_radial_angular_bins(neighbor_vectors[i], 
                                                                       cell_orientations[i],
                                                            angular_bins=angular_bins, radial_bins=radial_bins)
        radial_indices_all[i] = radial_indices
        angular_indices_all[i] = angular_indices

    ##############################

    # neighbor_info = {'neighbors': neighbors,
    #  'neighbor_vectors': neighbor_vectors,
    #  'radial_indices': radial_indices_all,
    #  'angular_indices': angular_indices_all
    # }

    # save_pickle(neighbor_info, get_cell_data_filepath('neighbor_info', stack, sec, ext='pkl'))

    # contours = {cnt['name']: cnt['vertices'] for cnt_id, cnt in contour_df[contour_df['section'] == sec].iterrows()}

    #############################
    # Compute histograms
    #############################

    all_cell_size_histogram_all_labels_all_regions = defaultdict(list)
    all_cell_size_weighted_histogram_all_labels_all_regions = defaultdict(list)
    large_cell_size_histogram_all_labels_all_regions = defaultdict(list)

    large_cell_orientation_histogram_all_labels_all_regions = defaultdict(list)

    neighbor_distance_histogram_all_labels_all_regions = defaultdict(list)
    neighbor_direction_histogram_all_labels_all_regions = defaultdict(list)

    large_cell_graph_neighbor_distance_histogram_all_labels_all_regions = defaultdict(list)
    large_cell_graph_neighbor_direction_histogram_all_labels_all_regions = defaultdict(list)

    large_small_cell_graph_neighbor_distance_histogram_all_labels_all_regions = defaultdict(list)
    large_small_cell_graph_neighbor_direction_histogram_all_labels_all_regions = defaultdict(list)

    #######################################################
    # Identify large and small cells
    ######################################################

    for name_u, cnts in training_contours_all_labels_per_section[(stack, sec)].iteritems():
        
        if convert_to_original_name(name_u) != 'RtTg':
            continue

        for cnt in cnts:

            # Get large cells

            large_cell_is_inside = Path(cnt.astype(np.int)).contains_points(large_cell_centroids)
            large_cell_indices_inside = large_cell_indices[large_cell_is_inside]

    #         print '%d large cells are identified in %s.' % (len(large_cell_indices_inside), name_u)

            # Small cells

            small_cell_indices = np.array(list(set(range(cell_numbers)) - set(large_cell_indices.tolist())))
            small_cell_centroids = cell_centroids[small_cell_indices]
            small_cell_is_inside = Path(cnt.astype(np.int)).contains_points(small_cell_centroids)
            small_cell_indices_inside = small_cell_indices[small_cell_is_inside]

    #         print '%d small cells are identified in %s.' % (len(small_cell_indices_inside), name_u)

            #################################
            # Compute cell size distribution
            #################################

            large_cell_size_inside_histogram, _ = np.histogram([cell_sizes[i] for i in large_cell_indices_inside], bins=size_bins)
            large_cell_size_histogram_all_labels_all_regions[name_u].append(large_cell_size_inside_histogram)

            all_cell_size_inside_histogram, _ = np.histogram([cell_sizes[i] for i in np.r_[large_cell_indices_inside, small_cell_indices_inside]], 
                                                   bins=size_bins)
            all_cell_size_histogram_all_labels_all_regions[name_u].append(all_cell_size_inside_histogram)


            all_cell_size_weighted_inside_histogram, _ = np.histogram([cell_sizes[i] for i in np.r_[large_cell_indices_inside, small_cell_indices_inside]], 
                                               bins=size_bins, 
                                                         weights=[cell_sizes[i] for i in np.r_[large_cell_indices_inside, small_cell_indices_inside]])
            all_cell_size_weighted_histogram_all_labels_all_regions[name_u].append(all_cell_size_weighted_inside_histogram)


            #########################################
            # Compute cell orientation distribution
            #########################################

            large_cell_orientation_inside_histogram, _ = np.histogram([cell_orientations[i] for i in large_cell_indices_inside], bins=orientation_bins)
        #     print 'large_cell_orientation_inside_histogram:', large_cell_orientation_inside_histogram

            large_cell_orientation_histogram_all_labels_all_regions[name_u].append(large_cell_orientation_inside_histogram)

            ##########################################
            # Construct graph
            # Start from large cells, link to all cells
            ##########################################

            import networkx as nx
            g = nx.Graph()

            for source_sectionwise_idx in large_cell_indices_inside:

                if len(neighbors[source_sectionwise_idx]) == 0:
                    continue

                neighbor_masks = np.array([cells[i_sectionwise_idx]
                                  for i_sectionwise_idx in neighbors[source_sectionwise_idx]])

                jacs, _ = compute_jaccard_x_vs_list(cells[source_sectionwise_idx], 
                                                neighbor_masks,
                                                x_h=cells_h[source_sectionwise_idx],
                                                 x_v=cells_v[source_sectionwise_idx],
                                                 x_d=cells_d[source_sectionwise_idx])

                for i_sectionwise_idx, vec, jac in zip(neighbors[source_sectionwise_idx], neighbor_vectors[source_sectionwise_idx], jacs):
                    length = np.sqrt(np.sum(vec**2))
                    direction = np.arctan2(vec[1], vec[0])

                    # Force into between -np.pi/2 and np.pi/2
                    if direction > np.pi/2:
                        direction = direction - np.pi
                    elif direction < -np.pi/2:
                        direction = direction + np.pi

                    size_diff = np.abs(cell_sizes[source_sectionwise_idx] - cell_sizes[i_sectionwise_idx])

                    orientation_diff = cell_orientations[source_sectionwise_idx] - cell_orientations[i_sectionwise_idx]
                    if orientation_diff > np.pi/2:
                        orientation_diff -= np.pi
                    elif orientation_diff < -np.pi/2:
                        orientation_diff += np.pi

                    g.add_edge(source_sectionwise_idx, i_sectionwise_idx, weight=1, length=length, direction=direction, 
                               orientation_diff=orientation_diff,
                              size_diff=size_diff,
                              jaccard=jac)

            ##########################################
            # Collect edge attributes
            ##########################################

            all_edge_length = []
            all_edge_direction = []
            for u, v in g.edges_iter():
            #     print u, v
                ed = g.get_edge_data(u, v)
                all_edge_length.append(ed['length'])
                all_edge_direction.append(ed['direction'])

            ##########################################
            # Compute neighbor distance histogram
            ##########################################

        #     print 'min', np.min(all_edge_length), 'max', np.max(all_edge_length)

            edge_length_histogram, _ = np.histogram(all_edge_length, bins=edge_length_bins)
            neighbor_distance_histogram_all_labels_all_regions[name_u].append(edge_length_histogram)

            ##########################################
            # Compute neighbor direction histogram
            ##########################################

            edge_direction_histogram, _ = np.histogram(all_edge_direction, bins=edge_direction_bins)
            neighbor_direction_histogram_all_labels_all_regions[name_u].append(edge_direction_histogram)

            ################################
            # Subgraph between large cells.
            ################################

            large_cell_subgraph = g.subgraph(large_cell_indices_inside)

            all_large_cell_graph_edge_length = []
            all_large_cell_graph_edge_direction = []
            for u, v in large_cell_subgraph.edges_iter():
                ed = large_cell_subgraph.get_edge_data(u, v)
                all_large_cell_graph_edge_length.append(ed['length'])
                all_large_cell_graph_edge_direction.append(ed['direction'])

            ##########################################
            # Compute neighbor distance histogram
            ##########################################

        #     print 'min', np.min(all_edge_length), 'max', np.max(all_edge_length)

            edge_length_histogram, _ = np.histogram(all_large_cell_graph_edge_length, bins=edge_length_bins)
            large_cell_graph_neighbor_distance_histogram_all_labels_all_regions[name_u].append(edge_length_histogram)

            ##########################################
            # Compute neighbor direction histogram
            ##########################################

            edge_direction_histogram, _ = np.histogram(all_large_cell_graph_edge_direction, bins=edge_direction_bins)
            large_cell_graph_neighbor_direction_histogram_all_labels_all_regions[name_u].append(edge_direction_histogram)

            ##########################################
            # Large/small cells graph
            ##########################################

            all_large_small_cell_graph_edge_length = []
            all_large_small_cell_graph_edge_direction = []

            for u, v in g.edges_iter():
                if (u in large_cell_indices_inside and v in small_cell_indices_inside) or \
                (v in large_cell_indices_inside and u in small_cell_indices_inside):
                    ed = g.get_edge_data(u,v)
                    all_large_small_cell_graph_edge_length.append(ed['length'])
                    all_large_small_cell_graph_edge_direction.append(ed['direction'])

            edge_length_histogram, _ = np.histogram(all_large_small_cell_graph_edge_length, bins=edge_length_bins)
            large_small_cell_graph_neighbor_distance_histogram_all_labels_all_regions[name_u].append(edge_length_histogram)

            edge_direction_histogram, _ = np.histogram(all_large_small_cell_graph_edge_direction, bins=edge_direction_bins)
            large_small_cell_graph_neighbor_direction_histogram_all_labels_all_regions[name_u].append(edge_direction_histogram)
            
    all_cell_size_histogram_all_labels_all_regions_per_section[(stack, sec)] = all_cell_size_histogram_all_labels_all_regions
    all_cell_size_weighted_histogram_all_labels_all_regions_per_section[(stack, sec)] = all_cell_size_weighted_histogram_all_labels_all_regions
    large_cell_size_histogram_all_labels_all_regions_per_section[(stack, sec)] = large_cell_size_histogram_all_labels_all_regions

    large_cell_orientation_histogram_all_labels_all_regions_per_section[(stack, sec)] = large_cell_orientation_histogram_all_labels_all_regions
    neighbor_distance_histogram_all_labels_all_regions_per_section[(stack, sec)] = neighbor_distance_histogram_all_labels_all_regions
    neighbor_direction_histogram_all_labels_all_regions_per_section[(stack, sec)] = neighbor_direction_histogram_all_labels_all_regions

    large_cell_graph_neighbor_distance_histogram_all_labels_all_regions_per_section[(stack, sec)] = large_cell_graph_neighbor_distance_histogram_all_labels_all_regions
    large_cell_graph_neighbor_direction_histogram_all_labels_all_regions_per_section[(stack, sec)] = large_cell_graph_neighbor_direction_histogram_all_labels_all_regions

    large_small_cell_graph_neighbor_distance_histogram_all_labels_all_regions_per_section[(stack, sec)] = large_small_cell_graph_neighbor_distance_histogram_all_labels_all_regions
    large_small_cell_graph_neighbor_direction_histogram_all_labels_all_regions_per_section[(stack, sec)] = large_small_cell_graph_neighbor_direction_histogram_all_labels_all_regions


In [ ]:
all_cell_size_histogram_all_labels_all_regions_all_sections = defaultdict(list)
for (st, sec), hists in all_cell_size_histogram_all_labels_all_regions_per_section.iteritems():
    for label, hs in hists.iteritems():
        all_cell_size_histogram_all_labels_all_regions_all_sections[label].append(hs)
        
all_cell_size_histogram_all_labels_all_regions_all_sections = {label: np.concatenate(list_of_hists) 
    for label, list_of_hists in all_cell_size_histogram_all_labels_all_regions_all_sections.iteritems()}

In [ ]:
large_cell_orientation_histogram_all_labels_all_regions_all_sections = defaultdict(list)
for (st, sec), hists in large_cell_orientation_histogram_all_labels_all_regions_per_section.iteritems():
    for label, hs in hists.iteritems():
        large_cell_orientation_histogram_all_labels_all_regions_all_sections[label].append(hs)
        
large_cell_orientation_histogram_all_labels_all_regions_all_sections = {label: np.concatenate(list_of_hists) 
    for label, list_of_hists in large_cell_orientation_histogram_all_labels_all_regions_all_sections.iteritems()}

In [ ]:
# plot_histograms(large_cell_orientation_histogram_all_labels_all_regions_all_sections['RtTg_surround_500_Pn'][:20], 
#                 bins=orientation_bins[:-1], cellsize=(2, 1.5), normalize=True, ncols=8)

# plot_histograms(large_cell_orientation_histogram_all_labels_all_regions_all_sections['RtTg'][:20], 
#                 bins=orientation_bins[:-1], cellsize=(2, 1.5), normalize=True, ncols=8)

In [ ]:
train_feat_dir = create_if_not_exists(os.path.join(CELL_FEATURES_CLF_ROOTDIR, 'training_features/'))

In [ ]:
def normalize_rows(h):
    return h/h.sum(axis=1).astype(np.float)[:,None]

In [ ]:
# Save training features
for name in large_cell_orientation_histogram_all_labels_all_regions_all_sections.iterkeys():
    
#     features = [np.concatenate(all_cell_size_histogram_all_labels_all_regions_all_sections[i],
#                     large_cell_orientation_histogram_all_labels_all_regions_all_sections[i])
#      for i in range(len(large_cell_orientation_histogram_all_labels_all_regions_all_sections[name]))]

    h1 = normalize_rows(all_cell_size_histogram_all_labels_all_regions_all_sections[name])
    h2 = normalize_rows(large_cell_orientation_histogram_all_labels_all_regions_all_sections[name])
    features = np.hstack([h1, h2])
                           
    bp.pack_ndarray_file(features, os.path.join(train_feat_dir, name + '.bp'))

# One stack - whole structure

In [ ]:
stack = 'MD589'

contour_df, _ = DataManager.load_annotation_v3(stack=stack)
from annotation_utilities import *
contour_df = convert_annotation_v3_original_to_aligned_cropped(contour_df, stack)

In [ ]:
sec = 148

sys.stderr.write('Processing section %d...\n' % sec)

cell_orientations = load_cell_data('orientation', stack, sec)
cell_orientations = np.array(map(normalize_angle, cell_orientations))

cell_centroids = load_cell_data('centroid', stack, sec)
cell_numbers = cell_centroids.shape[0]

def flatten_cells(what, sec):
    cells = load_cell_data(what, stack, sec)
    cells = cells.reshape((cells.shape[0], -1))
    return cells

cells = flatten_cells('cells_aligned_padded', sec)
cells_h = flatten_cells('cells_aligned_padded_h', sec)
cells_v = flatten_cells('cells_aligned_padded_v', sec)
cells_d = flatten_cells('cells_aligned_padded_d', sec)

cell_sizes = np.sum(cells, axis=1)

large_cell_threshold = 163
large_cell_indices = np.where(cell_sizes > large_cell_threshold)[0]
large_cell_centroids = cell_centroids[large_cell_indices]

#####################################
# Find each LARGE cell's neighbors
#####################################

from scipy.spatial.kdtree import KDTree
from scipy.spatial.distance import cdist, pdist

t = time.time()

tree = KDTree(cell_centroids)

pool = Pool(12)
neighbors = pool.map(lambda i: list(set(tree.query_ball_point(cell_centroids[i], r=100)) - {i}),
                     large_cell_indices)
pool.terminate()
pool.join()

sys.stderr.write('Neighbor search: %.2f seconds\n' % (time.time()-t)) # 10 seconds

neighbors = dict(zip(large_cell_indices, neighbors))

# Compute neighbot vectors

neighbor_vectors = {i: cell_centroids[i] - cell_centroids[nns] 
                    for i, nns in neighbors.iteritems()}

# Binning each cell's neighbors

n_radial = 4
# radial_bins = np.logspace(0, 2, 10, base=10)
radial_bins = np.linspace(0, 100, n_radial+1)

n_angular = 8
angular_bins = np.linspace(-np.pi, np.pi, n_angular+1)

radial_indices_all = {}
angular_indices_all = {}
for i in neighbor_vectors.iterkeys():
    radial_indices, angular_indices = allocate_radial_angular_bins(neighbor_vectors[i], 
                                                                   cell_orientations[i],
                                                        angular_bins=angular_bins, radial_bins=radial_bins)
    radial_indices_all[i] = radial_indices
    angular_indices_all[i] = angular_indices

##############################

neighbor_info = {'neighbors': neighbors,
 'neighbor_vectors': neighbor_vectors,
 'radial_indices': radial_indices_all,
 'angular_indices': angular_indices_all
}

save_pickle(neighbor_info, get_cell_data_filepath('neighbor_info', stack, sec, ext='pkl'))

#     neighbors_all_sections[sec] = neighbors
#     neighbor_vectors_all_sections[sec] = neighbor_vectors
#     radial_indices_all_sections[sec] = radial_indices_all
#     angular_indices_all_sections[sec] = angular_indices_all

contours = {cnt['name']: cnt['vertices'] for cnt_id, cnt in contour_df[contour_df['section'] == sec].iterrows()}

#############################
# Compute histograms
#############################

all_cell_size_histogram_all_regions = {}
all_cell_size_weighted_histogram_all_regions = {}
large_cell_size_histogram_all_regions = {}

large_cell_orientation_histogram_all_regions = {}

neighbor_distance_histogram_all_regions = {}
neighbor_direction_histogram_all_regions = {}

large_cell_graph_neighbor_distance_histogram_all_regions = {}
large_cell_graph_neighbor_direction_histogram_all_regions = {}

large_small_cell_graph_neighbor_distance_histogram_all_regions = {}
large_small_cell_graph_neighbor_direction_histogram_all_regions = {}

#######################################################
# Identify large and small cells
######################################################

for name_u, cnt in contours.iteritems():
# for name_u, cnt in [('12N', contours['12N'])]:
# name_u = '7N'
# cnt = contours[name_u]

    # Get large cells

    large_cell_is_inside = Path(cnt.astype(np.int)).contains_points(large_cell_centroids)
    large_cell_indices_inside = large_cell_indices[large_cell_is_inside]

    print '%d large cells are identified in %s.' % (len(large_cell_indices_inside), name_u)

    # Small cells

    small_cell_indices = np.array(list(set(range(cell_numbers)) - set(large_cell_indices.tolist())))
    small_cell_centroids = cell_centroids[small_cell_indices]
    small_cell_is_inside = Path(cnt.astype(np.int)).contains_points(small_cell_centroids)
    small_cell_indices_inside = small_cell_indices[small_cell_is_inside]

    print '%d small cells are identified in %s.' % (len(small_cell_indices_inside), name_u)

    #################################
    # Compute cell size distribution
    #################################

    size_bins = np.r_[np.linspace(0, 3000, 10), np.inf]

    large_cell_size_inside_histogram, _ = np.histogram([cell_sizes[i] for i in large_cell_indices_inside], bins=size_bins)
#     print 'large_cell_size_inside_histogram:', large_cell_size_inside_histogram
    large_cell_size_histogram_all_regions[name_u] = large_cell_size_inside_histogram

    all_cell_size_inside_histogram, _ = np.histogram([cell_sizes[i] for i in np.r_[large_cell_indices_inside, small_cell_indices_inside]], 
                                           bins=size_bins)
    #     print 'all_cell_size_inside_histogram:', all_cell_size_inside_histogram
    all_cell_size_histogram_all_regions[name_u] = all_cell_size_inside_histogram    


    all_cell_size_weighted_inside_histogram, _ = np.histogram([cell_sizes[i] for i in np.r_[large_cell_indices_inside, small_cell_indices_inside]], 
                                       bins=size_bins, 
                                                 weights=[cell_sizes[i] for i in np.r_[large_cell_indices_inside, small_cell_indices_inside]])
    all_cell_size_weighted_histogram_all_regions[name_u] = all_cell_size_weighted_inside_histogram    


    #########################################
    # Compute cell orientation distribution
    #########################################

    n_orientation_bins = 10
    orientation_bins = np.linspace(-np.pi/2, np.pi/2, n_orientation_bins+1)

    large_cell_orientation_inside_histogram, _ = np.histogram([cell_orientations[i] for i in large_cell_indices_inside], bins=orientation_bins)
#     print 'large_cell_orientation_inside_histogram:', large_cell_orientation_inside_histogram

    large_cell_orientation_histogram_all_regions[name_u] = large_cell_orientation_inside_histogram

    ##########################################
    # Construct graph
    # Start from large cells, link to all cells
    ##########################################

    import networkx as nx
    g = nx.Graph()

    for source_sectionwise_idx in large_cell_indices_inside:

        if len(neighbors[source_sectionwise_idx]) == 0:
            continue

        neighbor_masks = np.array([cells[i_sectionwise_idx]
                          for i_sectionwise_idx in neighbors[source_sectionwise_idx]])

        jacs, _ = compute_jaccard_x_vs_list(cells[source_sectionwise_idx], 
                                        neighbor_masks,
                                        x_h=cells_h[source_sectionwise_idx],
                                         x_v=cells_v[source_sectionwise_idx],
                                         x_d=cells_d[source_sectionwise_idx])

        for i_sectionwise_idx, vec, jac in zip(neighbors[source_sectionwise_idx], neighbor_vectors[source_sectionwise_idx], jacs):
            length = np.sqrt(np.sum(vec**2))
            direction = np.arctan2(vec[1], vec[0])

            # Force into between -np.pi/2 and np.pi/2
            if direction > np.pi/2:
                direction = direction - np.pi
            elif direction < -np.pi/2:
                direction = direction + np.pi

            size_diff = np.abs(cell_sizes[source_sectionwise_idx] - cell_sizes[i_sectionwise_idx])

            orientation_diff = cell_orientations[source_sectionwise_idx] - cell_orientations[i_sectionwise_idx]
            if orientation_diff > np.pi/2:
                orientation_diff -= np.pi
            elif orientation_diff < -np.pi/2:
                orientation_diff += np.pi

            g.add_edge(source_sectionwise_idx, i_sectionwise_idx, weight=1, length=length, direction=direction, 
                       orientation_diff=orientation_diff,
                      size_diff=size_diff,
                      jaccard=jac)

    ##########################################
    # Collect edge attributes
    ##########################################

    all_edge_length = []
    all_edge_direction = []
    for u, v in g.edges_iter():
    #     print u, v
        ed = g.get_edge_data(u, v)
        all_edge_length.append(ed['length'])
        all_edge_direction.append(ed['direction'])

    ##########################################
    # Compute neighbor distance histogram
    ##########################################

#     print 'min', np.min(all_edge_length), 'max', np.max(all_edge_length)

    n_edge_length_bins = 10
    edge_length_bins = np.r_[np.linspace(0, 100, n_edge_length_bins), np.inf]
    edge_length_histogram, _ = np.histogram(all_edge_length, bins=edge_length_bins)

    neighbor_distance_histogram_all_regions[name_u] = edge_length_histogram

    ##########################################
    # Compute neighbor direction histogram
    ##########################################

    n_edge_direction_bins = 10
    edge_direction_bins = np.linspace(-np.pi/2, np.pi/2, n_edge_direction_bins+1)
    edge_direction_histogram, _ = np.histogram(all_edge_direction, bins=edge_direction_bins)

    neighbor_direction_histogram_all_regions[name_u] = edge_direction_histogram

    ################################
    # Subgraph between large cells.
    ################################

    large_cell_subgraph = g.subgraph(large_cell_indices_inside)

    all_large_cell_graph_edge_length = []
    all_large_cell_graph_edge_direction = []
    for u, v in large_cell_subgraph.edges_iter():
        ed = large_cell_subgraph.get_edge_data(u, v)
        all_large_cell_graph_edge_length.append(ed['length'])
        all_large_cell_graph_edge_direction.append(ed['direction'])

    ##########################################
    # Compute neighbor distance histogram
    ##########################################

#     print 'min', np.min(all_edge_length), 'max', np.max(all_edge_length)

    edge_length_histogram, _ = np.histogram(all_large_cell_graph_edge_length, bins=edge_length_bins)
    large_cell_graph_neighbor_distance_histogram_all_regions[name_u] = edge_length_histogram

    ##########################################
    # Compute neighbor direction histogram
    ##########################################

    edge_direction_histogram, _ = np.histogram(all_large_cell_graph_edge_direction, bins=edge_direction_bins)
    large_cell_graph_neighbor_direction_histogram_all_regions[name_u] = edge_direction_histogram

    ##########################################
    # Large/small cells graph
    ##########################################

    all_large_small_cell_graph_edge_length = []
    all_large_small_cell_graph_edge_direction = []

    for u, v in g.edges_iter():
        if (u in large_cell_indices_inside and v in small_cell_indices_inside) or \
        (v in large_cell_indices_inside and u in small_cell_indices_inside):
            ed = g.get_edge_data(u,v)
            all_large_small_cell_graph_edge_length.append(ed['length'])
            all_large_small_cell_graph_edge_direction.append(ed['direction'])

    edge_length_histogram, _ = np.histogram(all_large_small_cell_graph_edge_length, bins=edge_length_bins)
    large_small_cell_graph_neighbor_distance_histogram_all_regions[name_u] = edge_length_histogram

    edge_direction_histogram, _ = np.histogram(all_large_small_cell_graph_edge_direction, bins=edge_direction_bins)
    large_small_cell_graph_neighbor_direction_histogram_all_regions[name_u] = edge_direction_histogram

# All stacks

In [ ]:
all_cell_size_histogram_all_regions_all_sections = {}
all_cell_size_weighted_histogram_all_regions_all_sections = {}
large_cell_size_histogram_all_regions_all_sections = {}

large_cell_orientation_histogram_all_regions_all_sections = {}

neighbor_distance_histogram_all_regions_all_sections = {}
neighbor_direction_histogram_all_regions_all_sections = {}

large_cell_graph_neighbor_distance_histogram_all_regions_all_sections = {}
large_cell_graph_neighbor_direction_histogram_all_regions_all_sections = {}

large_small_cell_graph_neighbor_distance_histogram_all_regions_all_sections = {}
large_small_cell_graph_neighbor_direction_histogram_all_regions_all_sections = {}

In [ ]:
# neighbors_all_sections = {}
# neighbor_vectors_all_sections = {}
# radial_indices_all_sections = {}
# angular_indices_all_sections = {}

In [ ]:
first_bs_section, last_bs_section = metadata_cache['section_limits'][stack]

In [ ]:
bar = show_progress_bar(first_bs_section, last_bs_section)

for sec in range(first_bs_section, last_bs_section+1):
# for sec in [148]:
    
    sys.stderr.write('Processing section %d...\n' % sec)
    
    bar.value = sec
    
    if metadata_cache['sections_to_filenames'][stack][sec] in ['Placeholder', 'Rescan', 'Nonexisting']:
        continue
    
    cell_orientations = load_cell_data('orientation', stack, sec)
    cell_orientations = np.array(map(normalize_angle, cell_orientations))

    cell_centroids = load_cell_data('centroid', stack, sec)
    cell_numbers = cell_centroids.shape[0]

    def flatten_cells(what, sec):
        cells = load_cell_data(what, stack, sec)
        cells = cells.reshape((cells.shape[0], -1))
        return cells
    
    cells = flatten_cells('cells_aligned_padded', sec)
    cells_h = flatten_cells('cells_aligned_padded_h', sec)
    cells_v = flatten_cells('cells_aligned_padded_v', sec)
    cells_d = flatten_cells('cells_aligned_padded_d', sec)
    
    cell_sizes = np.sum(cells, axis=1)

    large_cell_threshold = 163
    large_cell_indices = np.where(cell_sizes > large_cell_threshold)[0]
    
    #####################################
    # Find each LARGE cell's neighbors
    #####################################
    
    from scipy.spatial.kdtree import KDTree
    from scipy.spatial.distance import cdist, pdist

    t = time.time()

    tree = KDTree(cell_centroids)

    pool = Pool(12)
    neighbors = pool.map(lambda i: list(set(tree.query_ball_point(cell_centroids[i], r=100)) - {i}),
                         large_cell_indices)
    pool.terminate()
    pool.join()

    sys.stderr.write('Neighbor search: %.2f seconds\n' % (time.time()-t)) # 10 seconds

    neighbors = dict(zip(large_cell_indices, neighbors))

    # Compute neighbot vectors

    neighbor_vectors = {i: cell_centroids[i] - cell_centroids[nns] 
                        for i, nns in neighbors.iteritems()}

    # Binning each cell's neighbors

    n_radial = 4
    # radial_bins = np.logspace(0, 2, 10, base=10)
    radial_bins = np.linspace(0, 100, n_radial+1)

    n_angular = 8
    angular_bins = np.linspace(-np.pi, np.pi, n_angular+1)

    radial_indices_all = {}
    angular_indices_all = {}
    for i in neighbor_vectors.iterkeys():
        radial_indices, angular_indices = allocate_radial_angular_bins(neighbor_vectors[i], 
                                                                       cell_orientations[i],
                                                            angular_bins=angular_bins, radial_bins=radial_bins)
        radial_indices_all[i] = radial_indices
        angular_indices_all[i] = angular_indices
    
    ##############################
    
    neighbor_info = {'neighbors': neighbors,
     'neighbor_vectors': neighbor_vectors,
     'radial_indices': radial_indices_all,
     'angular_indices': angular_indices_all
    }
    
    save_pickle(neighbor_info, get_cell_data_filepath('neighbor_info', stack, sec, ext='pkl'))
    
#     neighbors_all_sections[sec] = neighbors
#     neighbor_vectors_all_sections[sec] = neighbor_vectors
#     radial_indices_all_sections[sec] = radial_indices_all
#     angular_indices_all_sections[sec] = angular_indices_all

    contours = {cnt['name']: cnt['vertices'] for cnt_id, cnt in contour_df[contour_df['section'] == sec].iterrows()}

    #############################
    # Compute histograms
    #############################
    
#     all_cell_size_histogram_all_regions = {}
#     all_cell_size_weighted_histogram_all_regions = {}
#     large_cell_size_histogram_all_regions = {}

#     large_cell_orientation_histogram_all_regions = {}

#     neighbor_distance_histogram_all_regions = {}
#     neighbor_direction_histogram_all_regions = {}

#     large_cell_graph_neighbor_distance_histogram_all_regions = {}
#     large_cell_graph_neighbor_direction_histogram_all_regions = {}

#     large_small_cell_graph_neighbor_distance_histogram_all_regions = {}
#     large_small_cell_graph_neighbor_direction_histogram_all_regions = {}
    
    #######################################################
    # Identify large and small cells
    ######################################################

    large_cell_indices = large_cell_indices
    large_cell_centroids = cell_centroids[large_cell_indices]

    for name_u, cnt in contours.iteritems():
    # for name_u, cnt in [('12N', contours['12N'])]:
    # name_u = '7N'
    # cnt = contours[name_u]

        # Get large cells

        large_cell_is_inside = Path(cnt.astype(np.int)).contains_points(large_cell_centroids)
        large_cell_indices_inside = large_cell_indices[large_cell_is_inside]

        print '%d large cells are identified in %s.' % (len(large_cell_indices_inside), name_u)

        # Small cells

        small_cell_indices = np.array(list(set(range(cell_numbers)) - set(large_cell_indices.tolist())))
        small_cell_centroids = cell_centroids[small_cell_indices]
        small_cell_is_inside = Path(cnt.astype(np.int)).contains_points(small_cell_centroids)
        small_cell_indices_inside = small_cell_indices[small_cell_is_inside]

        print '%d small cells are identified in %s.' % (len(small_cell_indices_inside), name_u)

        #################################
        # Compute cell size distribution
        #################################

        size_bins = np.r_[np.linspace(0, 3000, 10), np.inf]

        large_cell_size_inside_histogram, _ = np.histogram([cell_sizes[i] for i in large_cell_indices_inside], bins=size_bins)
    #     print 'large_cell_size_inside_histogram:', large_cell_size_inside_histogram
        large_cell_size_histogram_all_regions[name_u] = large_cell_size_inside_histogram

        all_cell_size_inside_histogram, _ = np.histogram([cell_sizes[i] for i in np.r_[large_cell_indices_inside, small_cell_indices_inside]], 
                                               bins=size_bins)
        #     print 'all_cell_size_inside_histogram:', all_cell_size_inside_histogram
        all_cell_size_histogram_all_regions[name_u] = all_cell_size_inside_histogram    


        all_cell_size_weighted_inside_histogram, _ = np.histogram([cell_sizes[i] for i in np.r_[large_cell_indices_inside, small_cell_indices_inside]], 
                                           bins=size_bins, 
                                                     weights=[cell_sizes[i] for i in np.r_[large_cell_indices_inside, small_cell_indices_inside]])
        all_cell_size_weighted_histogram_all_regions[name_u] = all_cell_size_weighted_inside_histogram    


        #########################################
        # Compute cell orientation distribution
        #########################################

        n_orientation_bins = 10
        orientation_bins = np.linspace(-np.pi/2, np.pi/2, n_orientation_bins+1)

        large_cell_orientation_inside_histogram, _ = np.histogram([cell_orientations[i] for i in large_cell_indices_inside], bins=orientation_bins)
    #     print 'large_cell_orientation_inside_histogram:', large_cell_orientation_inside_histogram

        large_cell_orientation_histogram_all_regions[name_u] = large_cell_orientation_inside_histogram

        ##########################################
        # Construct graph
        # Start from large cells, link to all cells
        ##########################################

        import networkx as nx
        g = nx.Graph()

        for source_sectionwise_idx in large_cell_indices_inside:
            
            if len(neighbors[source_sectionwise_idx]) == 0:
                continue

            neighbor_masks = np.array([cells[i_sectionwise_idx]
                              for i_sectionwise_idx in neighbors[source_sectionwise_idx]])

            jacs, _ = compute_jaccard_x_vs_list(cells[source_sectionwise_idx], 
                                            neighbor_masks,
                                            x_h=cells_h[source_sectionwise_idx],
                                             x_v=cells_v[source_sectionwise_idx],
                                             x_d=cells_d[source_sectionwise_idx])

            for i_sectionwise_idx, vec, jac in zip(neighbors[source_sectionwise_idx], neighbor_vectors[source_sectionwise_idx], jacs):
                length = np.sqrt(np.sum(vec**2))
                direction = np.arctan2(vec[1], vec[0])

                # Force into between -np.pi/2 and np.pi/2
                if direction > np.pi/2:
                    direction = direction - np.pi
                elif direction < -np.pi/2:
                    direction = direction + np.pi

                size_diff = np.abs(cell_sizes[source_sectionwise_idx] - cell_sizes[i_sectionwise_idx])
                
                orientation_diff = cell_orientations[source_sectionwise_idx] - cell_orientations[i_sectionwise_idx]
                if orientation_diff > np.pi/2:
                    orientation_diff -= np.pi
                elif orientation_diff < -np.pi/2:
                    orientation_diff += np.pi
                
                g.add_edge(source_sectionwise_idx, i_sectionwise_idx, weight=1, length=length, direction=direction, 
                           orientation_diff=orientation_diff,
                          size_diff=size_diff,
                          jaccard=jac)

        ##########################################
        # Collect edge attributes
        ##########################################

        all_edge_length = []
        all_edge_direction = []
        for u, v in g.edges_iter():
        #     print u, v
            ed = g.get_edge_data(u, v)
            all_edge_length.append(ed['length'])
            all_edge_direction.append(ed['direction'])

        ##########################################
        # Compute neighbor distance histogram
        ##########################################

    #     print 'min', np.min(all_edge_length), 'max', np.max(all_edge_length)

        n_edge_length_bins = 10
        edge_length_bins = np.r_[np.linspace(0, 100, n_edge_length_bins), np.inf]
        edge_length_histogram, _ = np.histogram(all_edge_length, bins=edge_length_bins)

        neighbor_distance_histogram_all_regions[name_u] = edge_length_histogram

        ##########################################
        # Compute neighbor direction histogram
        ##########################################

        n_edge_direction_bins = 10
        edge_direction_bins = np.linspace(-np.pi/2, np.pi/2, n_edge_direction_bins+1)
        edge_direction_histogram, _ = np.histogram(all_edge_direction, bins=edge_direction_bins)

        neighbor_direction_histogram_all_regions[name_u] = edge_direction_histogram

        ################################
        # Subgraph between large cells.
        ################################

        large_cell_subgraph = g.subgraph(large_cell_indices_inside)

        all_large_cell_graph_edge_length = []
        all_large_cell_graph_edge_direction = []
        for u, v in large_cell_subgraph.edges_iter():
            ed = large_cell_subgraph.get_edge_data(u, v)
            all_large_cell_graph_edge_length.append(ed['length'])
            all_large_cell_graph_edge_direction.append(ed['direction'])

        ##########################################
        # Compute neighbor distance histogram
        ##########################################

    #     print 'min', np.min(all_edge_length), 'max', np.max(all_edge_length)

        edge_length_histogram, _ = np.histogram(all_large_cell_graph_edge_length, bins=edge_length_bins)
        large_cell_graph_neighbor_distance_histogram_all_regions[name_u] = edge_length_histogram

        ##########################################
        # Compute neighbor direction histogram
        ##########################################

        edge_direction_histogram, _ = np.histogram(all_large_cell_graph_edge_direction, bins=edge_direction_bins)
        large_cell_graph_neighbor_direction_histogram_all_regions[name_u] = edge_direction_histogram

        ##########################################
        # Large/small cells graph
        ##########################################

        all_large_small_cell_graph_edge_length = []
        all_large_small_cell_graph_edge_direction = []

        for u, v in g.edges_iter():
            if (u in large_cell_indices_inside and v in small_cell_indices_inside) or \
            (v in large_cell_indices_inside and u in small_cell_indices_inside):
                ed = g.get_edge_data(u,v)
                all_large_small_cell_graph_edge_length.append(ed['length'])
                all_large_small_cell_graph_edge_direction.append(ed['direction'])

        edge_length_histogram, _ = np.histogram(all_large_small_cell_graph_edge_length, bins=edge_length_bins)
        large_small_cell_graph_neighbor_distance_histogram_all_regions[name_u] = edge_length_histogram

        edge_direction_histogram, _ = np.histogram(all_large_small_cell_graph_edge_direction, bins=edge_direction_bins)
        large_small_cell_graph_neighbor_direction_histogram_all_regions[name_u] = edge_direction_histogram
        
    ###########################################
    
    region_features = {'all_cell_size_hist': all_cell_size_histogram_all_regions,
    'all_cell_size_weighted_hist': all_cell_size_weighted_histogram_all_regions,
    'large_cell_size_hist': large_cell_size_histogram_all_regions,
    'large_cell_orientation_hist': large_cell_orientation_histogram_all_regions,
    'neighbor_distance_hist': neighbor_distance_histogram_all_regions,
    'neighbor_direction_hist': neighbor_direction_histogram_all_regions,
    'large_cell_graph_neighbor_distance_hist': large_cell_graph_neighbor_distance_histogram_all_regions,
    'large_cell_graph_neighbor_direction_hist': large_cell_graph_neighbor_direction_histogram_all_regions,
    'large_small_cell_graph_neighbor_distance_hist': large_small_cell_graph_neighbor_distance_histogram_all_regions,
    'large_small_cell_graph_neighbor_direction_hist': large_small_cell_graph_neighbor_direction_histogram_all_regions}
    
    save_pickle(region_features, get_cell_data_filepath('region_features', stack, sec, ext='pkl'))
    
#     save_pickle(all_cell_size_histogram_all_regions, get_cell_data_filepath('all_cell_size_hist', stack, sec, ext='pkl'))
#     save_pickle(all_cell_size_weighted_histogram_all_regions, get_cell_data_filepath('all_cell_size_weighted_hist', stack, sec, ext='pkl'))
#     save_pickle(large_cell_size_histogram_all_regions, get_cell_data_filepath('large_cell_size_hist', stack, sec, ext='pkl'))
#     save_pickle(large_cell_orientation_histogram_all_regions, get_cell_data_filepath('large_cell_orientation_hist', stack, sec, ext='pkl'))
#     save_pickle(neighbor_distance_histogram_all_regions, get_cell_data_filepath('neighbor_distance_hist', stack, sec, ext='pkl'))
#     save_pickle(neighbor_direction_histogram_all_regions, get_cell_data_filepath('neighbor_direction_hist', stack, sec, ext='pkl'))
#     save_pickle(large_cell_graph_neighbor_distance_histogram_all_regions, get_cell_data_filepath('large_cell_graph_neighbor_distance_hist', stack, sec, ext='pkl'))
#     save_pickle(large_cell_graph_neighbor_direction_histogram_all_regions, get_cell_data_filepath('large_cell_graph_neighbor_direction_hist', stack, sec, ext='pkl'))
#     save_pickle(large_small_cell_graph_neighbor_distance_histogram_all_regions, get_cell_data_filepath('large_small_cell_graph_neighbor_distance_hist', stack, sec, ext='pkl'))
#     save_pickle(large_small_cell_graph_neighbor_direction_histogram_all_regions, get_cell_data_filepath('large_small_cell_graph_neighbor_direction_hist', stack, sec, ext='pkl'))
    
#     all_cell_size_histogram_all_regions_all_sections[sec] = all_cell_size_histogram_all_regions
#     all_cell_size_weighted_histogram_all_regions_all_sections[sec] = all_cell_size_weighted_histogram_all_regions
#     large_cell_size_histogram_all_regions_all_sections[sec] = large_cell_size_histogram_all_regions

#     large_cell_orientation_histogram_all_regions_all_sections[sec] = large_cell_orientation_histogram_all_regions

#     neighbor_distance_histogram_all_regions_all_sections[sec] = neighbor_distance_histogram_all_regions
#     neighbor_direction_histogram_all_regions_all_sections[sec] = neighbor_direction_histogram_all_regions

#     large_cell_graph_neighbor_distance_histogram_all_regions_all_sections[sec] = large_cell_graph_neighbor_distance_histogram_all_regions
#     large_cell_graph_neighbor_direction_histogram_all_regions_all_sections[sec] = large_cell_graph_neighbor_direction_histogram_all_regions

#     large_small_cell_graph_neighbor_distance_histogram_all_regions_all_sections[sec] = large_small_cell_graph_neighbor_distance_histogram_all_regions
#     large_small_cell_graph_neighbor_direction_histogram_all_regions_all_sections[sec] = large_small_cell_graph_neighbor_direction_histogram_all_regions